In [5]:
from llama_index.readers.file import PDFReader
from llama_index.readers.file import PyMuPDFReader

from llama_index.core import SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.core import Document
from llama_index.core import VectorStoreIndex
from llama_index.core import QueryBundle
from llama_index.core import StorageContext

from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.schema import TextNode, NodeWithScore
from llama_index.core.storage.docstore import SimpleDocumentStore

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import json
import Stemmer

#模型下载
from modelscope import snapshot_download
import pandas as pd
from tqdm import tqdm

from pathlib import Path
import os
import pdfplumber

import numpy as np
from FlagEmbedding import FlagModel
from sentence_transformers import SentenceTransformer

from zhipuLLM import GLM
from llama_index.core import StorageContext, load_index_from_storage

from graph_rag import GraphRAGExtractor, GraphRAGStore
import re

In [6]:
model_dir = snapshot_download('AI-ModelScope/bge-large-zh-v1.5', cache_dir="./models")
rerank_dir = snapshot_download('AI-ModelScope/bge-reranker-v2-m3', cache_dir="./models")
# rerank_dir = snapshot_download('BAAI/bge-reranker-v2-minicpm-layerwise', cache_dir="./models")
# tokenizer = AutoTokenizer.from_pretrained(rerank_dir, use_fast=False, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(rerank_dir, trust_remote_code=True, torch_dtype=torch.bfloat16)
#rerank_dir = "./models/bge-reranker-v2-minicpm-layerwise"
reranker = SentenceTransformerRerank(model=rerank_dir, top_n=100)

2024-11-15 20:54:47,419 - modelscope - WARNING - Authentication has expired, please re-login with modelscope login --token "YOUR_SDK_TOKEN" if you need to access private models or datasets.


In [ ]:
import nest_asyncio
nest_asyncio.apply()

LLAMA_CLOUD_API_KEY="llx-0sSh6yWMfyrPI6sCnYbTPBqoWw63WidhqACRvlYF0VNIrJYh"
# bring in our LLAMA_CLOUD_API_KEY
from dotenv import load_dotenv
load_dotenv()

# bring in deps
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# set up parser
parser = LlamaParse(
    result_type="markdown",  # "markdown" and "text" are available
    language="ch_sim",
    api_key=LLAMA_CLOUD_API_KEY
)

# use SimpleDirectoryReader to parse our file
input_files=['/A_document/AF01.pdf']
input_dir = "./B_document"
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_dir=input_dir, file_extractor=file_extractor).load_data() #input_files=./A_documen/AF01.PDF

In [7]:
def doc_saver(documents, file_path):
    
    with open(file_path, 'w', encoding='utf-8') as file:
    # 遍历字典列表中的每一个字典
        for document in tqdm(documents):
            # 将字典转换为JSON字符串，并写入文件
            data_line = document.to_embedchain_format()
            file.write(json.dumps(data_line, ensure_ascii=False) + '\n')

        print(f"数据已成功写入 {file_path}")
        
def doc_loader(file_path):
    
    # 创建一个空列表来存储解析后的字典
    documents = []

    # 打开文件，使用with语句确保文件正确关闭
    with open(file_path, 'r', encoding='utf-8') as file:
        # 逐行读取文件内容
        for line in tqdm(file):
            # 去除行尾的换行符，并将JSON字符串解析为字典
            item = json.loads(line.strip())
            # 将字典添加到列表中
            doc = Document().from_embedchain_format(item)
            documents.append(doc)
            
    return documents

import glob
def read_data(folder_path):
    
        # 初始化一个空列表来存储所有文件的内容
    txt_files = glob.glob(os.path.join(folder_path, '*.txt'))
    txt_content_list = []
    
    # 遍历每个 .txt 文件
    for file_path in txt_files:
        with open(file_path, 'r', encoding='utf-8') as file:
            # 读取文件内容并添加到列表中
            content = file.read()
            txt_content_list.append(content)
    
    return txt_content_list

In [8]:
import re
def filter_text(text):
    # '\n', ' ' 删除
    text = text.replace(' ','')
    # 删除页码
    # 删除本文档为2024CCFBDC***
    head_pattern = '本文档为2024CCFBDCI比赛用语料的一部分。[\s\S]+仅允许在本次比赛中使用。'
    # news_pattern
    pattern1 = r"发布时间：[\s\S]+发布人：新闻宣传中心"
    pattern2 = r"发布时间：[\s\S]+发布人：新闻发布人"
    pattern3 =  r'发布时间：\d{4}年\d{1,2}月\d{1,2}日'
    news_pattern = head_pattern+'|'+pattern1+'|'+pattern2+'|'+pattern3
    text = re.sub(news_pattern,'',text)
    # report_pattern
    report_pattern1 = '第一节重要提示[\s\S]+本次利润分配方案尚需提交本公司股东大会审议。'
    report_pattern12 = '一重要提示[\s\S]+股东大会审议。'
    report_pattern13 = '一、重要提示[\s\S]+季度报告未经审计。'
    report_pattern2 = '本公司董事会及全体董事保证本公告内容不存在任何虚假记载、[\s\S]+季度财务报表是否经审计[\s\S]+□是√否'
    report_pattern3 = '中国联合网络通信股份有限公司（简称“公司”）董事会审计委员会根据相关法律法规、[\s\S]+汇报如下：'
    report_pattern = report_pattern1+'|'+report_pattern12+'|'+report_pattern13+'|'+report_pattern2+'|'+report_pattern3
    text = re.sub( report_pattern,'',text)
    
#     text = text.replace('(','（').replace(')','）')
#     first_line = text.split('\n')[0].strip()
#     if first_line.startswith('#No'):
#         process_text = text.replace(first_line ,'')
#         text = re.sub(r'^#(.*?)\n\n#(.*?)\n', r'#\1\2\n', process_text, flags=re.MULTILINE | re.DOTALL).strip()
#     else:
#         text = re.sub(r'^#(.*?)\n\n#(.*?)\n', r'#\1\2\n', text, flags=re.MULTILINE | re.DOTALL).strip()
#         # 使用正则表达式获取第一行内容
#     first_line = text.split('\n')[0].strip()
#     final_text = first_line + text.replace(first_line, '')
        
    return text

### load file

In [9]:
# c_texts = read_data("./datas/data2/output_txt")
# documents = [Document(text=filter_text(text)) for text in c_texts]
file_path_a = "doc.jsonl"
file_path_b = "doc_b.jsonl"
#doc_saver(documents, file_path)
documents = doc_loader(file_path_b) + doc_loader(file_path_a)

718it [00:00, 23595.26it/s]
1062it [00:00, 23941.69it/s]


In [10]:
from collections import defaultdict
import re

# 使用 defaultdict 来收集具有相同 file_name 的文档
document_dict = defaultdict(list)

for doc in documents:
    document_dict[doc.metadata['file_name']].append(doc)

# 合并具有相同 file_name 的文档
merged_documents = []
for file_name, docs in document_dict.items():
    merged_text = docs[0].text.strip()
    for i in range(1, len(docs)):
        if merged_text.endswith('。') or docs[i].text.startswith('#'):
        # 如果前一个字符串以句号结尾，或当前字符串以#开头，用两个换行符拼接
            merged_text += '\n\n' + docs[i].text.strip()
        else:
        # 否则直接拼接
             merged_text += docs[i].text.strip()
        # 合并文本内容
        
    publish_time_match = re.search(r'发布时间：(\d{4}-\d{2}-\d{2})', merged_text)
    publisher_match = re.search(r'发布人：(\S+)', merged_text)
    merged_text = filter_text(merged_text)
        
    # 使用第一个文档的元数据作为合并后的文档元数据
    merged_metadata = docs[0].metadata.copy()
    # 提取元数据key
    ori_metadata = [key for key in docs[0].metadata.keys()]
    # 提取发布时间和发布人
    #title_match = re.search(r'#\s*(.+?)\n', merged_text)
        
    merged_metadata['发布时间'] = publish_time_match.group(1) if publish_time_match else ''
    merged_metadata['发布人'] = publisher_match.group(1) if publisher_match else ''
        #merged_metadata['title'] = title_match.group(1) if title_match else ''
    
    merged_doc = Document(
        id_=docs[0].id_,
        embedding=None,
        metadata=merged_metadata,
        excluded_embed_metadata_keys=ori_metadata,
        excluded_llm_metadata_keys=ori_metadata,
        relationships={},
        text=merged_text,
        mimetype='text/plain',
        start_char_idx=None,
        end_char_idx=None,
        text_template='{metadata_str}\n\n{content}',
        metadata_template='{key}: {value}',
        metadata_seperator='\n'
    )
    merged_documents.append(merged_doc)
documents = merged_documents

In [11]:
import jieba
def chinese_tokenizer(text: str) -> list[str]:
    try:
        # 加载停用词
        stopwords = set()
        if os.path.exists('stopwords.txt'):
            with open('stopwords.txt', 'r', encoding='utf-8') as f:
                stopwords = set(line.strip() for line in f)

        # 分词并过滤停用词
        tokens = jieba.lcut(text)
        return [token for token in tokens if token not in stopwords]
    except Exception as e:
        logger.error(f"Error in tokenization: {e}")
        return jieba.lcut(text)  # 降级处理

In [12]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    BaseExtractor,
)
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai_like import OpenAILike as OpenAI
#from llama_index.llms.openllm import OpenLLM as OpenAI
#from llama_index.legacy.llms import OpenAILike as OpenAI
from llama_index.legacy.callbacks import CallbackManager
from llama_index.core import Settings
from dotenv import dotenv_values
import os
import nltk

config = dotenv_values(".env")
glm_api_key = config["glm_api_key"]
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

callback_manager = CallbackManager()

text_splitter = SentenceSplitter(chunk_size=516, chunk_overlap=128, tokenizer=chinese_tokenizer)

llm = OpenAI(
        model="glm-4-plus",
        api_base="https://open.bigmodel.cn/api/paas/v4",
        api_key=glm_api_key,
        callback_manager=callback_manager,
        is_chat_model=True
    )
Settings.llm = llm

QA_template = """\
以下是上下文信息：
{context_str}

请根据上述上下文信息，生成 {num_questions} 个问题。这些问题的具体答案应主要来源于上下文，而不是其他地方。
利用这些上下文信息，生成有深度和相关性的问题，这些问题能够由提供的上下文信息准确回答。
注意，请直接生成问题即可，无需额外解释。
"""

TITLE_NODE_TEMPLATE = """\
以下是上下文信息：
{context_str}

给出一个标题，该标题总结上下文中所有独特的实体、标题或主题。
注意，直接生成标题即可，无需任何额外解释。
标题:
"""

TITLE_COMBINE_TEMPLATE = """\
{context_str}

基于上述候选标题和内容，输出这份文档的综合标题。
注意，直接生成标题即可，无需任何额外解释。
标题:
"""

KEYWORD_EXTRACT_TEMPLATE = """
{context_str}
为这份文档提供{keywords}个独特的关键词,并以逗号(,)分隔。

示例:
关键词, 关键词, 关键词...
"""

extractors = [
    TitleExtractor(nodes=1, node_template=TITLE_NODE_TEMPLATE, combine_template=TITLE_COMBINE_TEMPLATE, llm=GLM()),
    QuestionsAnsweredExtractor(questions=3, prompt_template=QA_template,llm=GLM()),
    #EntityExtractor(model_name="span-marker", prediction_threshold=0.5, label_entities=True, device="cuda"),
    # SummaryExtractor(summaries=["prev", "self"], llm=llm),
    KeywordExtractor(keywords=10, prompt_template=KEYWORD_EXTRACT_TEMPLATE, llm=GLM()),
    # CustomExtractor()
]

transformations = [text_splitter] + extractors
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=transformations)

# uber_nodes = pipeline.run(documents=documents)
# base_index = VectorStoreIndex(uber_nodes)
# base_index.storage_context.persist(persist_dir="./storage")

/usr/local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in EntityExtractor has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [13]:
# 使用本地模型路径创建 HuggingFaceEmbedding 实例
embed_model = HuggingFaceEmbedding(
    model_name=model_dir,  # 使用本地模型路径
    cache_folder=model_dir,  # 指定缓存目录
    embed_batch_size=512
)

Settings.embed_model = embed_model

In [ ]:
from datetime import datetime

# 获取当前时间并格式化
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
persist_dir= './storage/20241110_192022' #f"./storage/{current_time}"

for docs in tqdm(documents):
    nodes = pipeline.run(documents=[docs])
    if os.path.exists(persist_dir):
        base_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=persist_dir)
        )
        base_index.insert_nodes(nodes)
    else:
        base_index = VectorStoreIndex(nodes)
    base_index.storage_context.persist(persist_dir=persist_dir)

1

In [6]:
# # 指定文件夹路径
# input_dir = "./A_document"

# # 初始化一个字典来存储每个 PDF 文件的文本
# pdf_texts = {}

# # 遍历文件夹中的所有文件
# for filename in tqdm(os.listdir(input_dir)):
#     if filename.endswith(".pdf"):
#         file_path = os.path.join(input_dir, filename)
        
#         # 打开 PDF 文件
#         with pdfplumber.open(file_path) as pdf:
#             whole_text = ''
#             # 遍历每一页
#             for page in pdf.pages:
#                 # 提取文本
#                 text = page.extract_text()
#                 if text:
#                     whole_text += text + '\n'
        
#             # 去掉第一句话，并将第二句话作为文章标题
#             sentences = whole_text.split('。')
#             if len(sentences) > 1:
#                 content = '。'.join(sentences[1:])  # 从第三句话开始拼接内容
#                 content = whole_text
#         # 存储文本
#         pdf_texts[filename] = whole_text

### Tokenizer

### Set embed_model and LLM

In [11]:
# 使用本地模型路径创建 HuggingFaceEmbedding 实例
embed_model = HuggingFaceEmbedding(
    model_name=model_dir,  # 使用本地模型路径
    cache_folder=model_dir,  # 指定缓存目录
    embed_batch_size=512
)

Settings.embed_model = embed_model

### Similarity

In [14]:
def cosine_similarity(vec_a, vec_b):
    dot_product = np.dot(vec_a, vec_b)
    norm_a = np.linalg.norm(vec_a)
    norm_b = np.linalg.norm(vec_b)
    return dot_product / (norm_a * norm_b)


def get_similarity(sentences_1, sentences_2):
    sentences_1 = sentences_1
    sentences_2 = sentences_2
    model = SentenceTransformer(model_dir)
    embeddings_1 = model.encode(sentences_1, normalize_embeddings=True)
    embeddings_2 = model.encode(sentences_2, normalize_embeddings=True)
    similarity = embeddings_1 @ embeddings_2.T
    return similarity


def get_scores(query, prompt):
    queries = [query]
    passages = prompt
    instruction = "为这个句子生成表示以用于检索相关文章："

    model = SentenceTransformer(model_dir)
    q_embeddings = model.encode([instruction+q for q in queries], normalize_embeddings=True)
    p_embeddings = model.encode(passages, normalize_embeddings=True)
    scores = q_embeddings @ p_embeddings.T
    return scores

### Base Indexing

In [15]:
import torch

torch.cuda.empty_cache()
## base_index
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import TokenTextSplitter

# text_splitter = TokenTextSplitter(
#     separator="。", chunk_size=1024, chunk_overlap=128
# )

text_splitter = SentenceSplitter(chunk_size=516, chunk_overlap=128, tokenizer=chinese_tokenizer)
base_nodes = text_splitter.get_nodes_from_documents(documents)
base_index = VectorStoreIndex(base_nodes)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.785 seconds.
Prefix dict has been built successfully.


### Auto_retriever_index

In [16]:
## Load index into vector index
torch.cuda.empty_cache()
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
from llama_index.core import VectorStoreIndex


node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes = [2048, 1024, 256])
nodes = node_parser.get_nodes_from_documents(documents)
leaf_nodes = get_leaf_nodes(nodes)
root_nodes = get_root_nodes(nodes)

auto_retriever_index = VectorStoreIndex(leaf_nodes)

docstore = SimpleDocumentStore()
# insert nodes into docstore
docstore.add_documents(nodes)
# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

### SentenceWindowIndexing

In [17]:
torch.cuda.empty_cache()
for d in documents:
    d.text = d.text.replace('。','. ')
    d.text = d.text.replace('！','! ')
    d.text = d.text.replace('？','? ')
    d.text = d.text.replace('，',', ')
    
from llama_index.core.node_parser import SentenceWindowNodeParser
 
#定义句子解析器
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

nodes = node_parser.get_nodes_from_documents(documents)
sentence_index = VectorStoreIndex(nodes)

### PropertyGraphIndex

In [ ]:
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.property_graph import (
    ImplicitPathExtractor,
    SimpleLLMPathExtractor,
)
#from llama_index.llms.dashscope import DashScope

import nest_asyncio
import os
nest_asyncio.apply()

#os.environ["DASHSCOPE_API_KEY"] = "c173736660e1fc3a20d7f2ad1f368064.ywcM3QDFttUwKpul"
# graph_index = PropertyGraphIndex.from_documents(
#     documents=documents,
#     llm=DashScope(
#         model = "qwen-plus-latest",
#         base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
#         api_key="sk-9fe47a41bf8742dc908ce651fd8261a2",
#         is_chat_model=True,
#     ),
#     embed_model=embed_model,
#     show_progress=True
# )
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=128, tokenizer=chinese_tokenizer)
extractors = [
    TitleExtractor(nodes=5, llm=llm),
    QuestionsAnsweredExtractor(questions=3, prompt_template=QA_template,llm=llm),
    KeywordExtractor(keywords=10, llm=llm),
]

transformations = [text_splitter] + extractors

graph_index = PropertyGraphIndex.from_documents(
    documents=documents,
    transformations=transformations,
    kg_extractors=[
        SimpleLLMPathExtractor(
            llm=llm,
            num_workers=5,
            max_paths_per_chunk=20,
        ),
    ],
    llm=llm,
    embed_model=embed_model,
    show_progress=True
)

# graph_index.storage_context.persist(persist_dir="./storage")
# graph_index.property_graph_store.save_networkx_graph(name="./中国联通.html")
# base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1",
# api_key = "sk-9fe47a41bf8742dc908ce651fd8261a2", # 随便填写，只是为了通过接口参数校验

In [18]:
persist_dir = './storage/20241110_192022'
graph_index = load_index_from_storage(StorageContext.from_defaults(persist_dir=persist_dir))

In [17]:
from graph_rag import KG_TRIPLET_EXTRACT_TMPL
from llama_index.core import PropertyGraphIndex

def parse_fn(response_str: str) -> any:
    entity_pattern = r'\("entity"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\)'
    relationship_pattern = r'\("relationship"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\$\$\$\$"(.+?)"\)'
    entities = re.findall(entity_pattern, response_str)
    relationships = re.findall(relationship_pattern, response_str)
    return entities, relationships


kg_extractor = GraphRAGExtractor(
    llm=llm,
    extract_prompt=KG_TRIPLET_EXTRACT_TMPL,
    max_paths_per_chunk=2,
    parse_fn=parse_fn,
)


graph_index = PropertyGraphIndex(
    nodes=uber_nodes,
    property_graph_store=GraphRAGStore(),
    kg_extractors=[kg_extractor],
    show_progress=True,
)

NameError: name 'uber_nodes' is not defined

### Load graph_index

In [ ]:
graph_index.storage_context.persist(persist_dir="./storage")
graph_index.property_graph_store.save_networkx_graph(name="./中国联通.html")
graph_index = load_index_from_storage(
    StorageContext.from_defaults(persist_dir="./storage")
)

### Retrievers

In [132]:
def window_retriever(query, topk):
    sentence_window_retriever = sentence_index.as_retriever(similarity_top_k=topk)
    nodes = sentence_window_retriever.retrieve(query)
    #answer = [node.metadata['window'] for node in nodes]

    return nodes #answer

def base_vector_retriever(query, topk):
    base_retriever = base_index.as_retriever(similarity_top_k=topk)
    nodes = base_retriever.retrieve(query)
    #answer = [node.text for node in nodes]

    return nodes #answer

def get_bm25_retriever(query, topk):
# We can pass in the index, docstore, or list of nodes to create the retriever
    bm25_retriever = BM25Retriever.from_defaults(
        nodes=base_nodes,
        similarity_top_k=topk,
        # Optional: We can pass in the stemmer and set the language for stopwords
        # This is important for removing stopwords and stemming the query + text
        # The default is english for both
        tokenizer=chinese_tokenizer
    )

    nodes = bm25_retriever.retrieve(query)
    #answer = [node.text for node in nodes]
    return nodes #answer

def auto_retriever(query, topk):
    base_retriever = auto_retriever_index.as_retriever(similarity_top_k=topk)
    auto_retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=False)
    
    nodes = auto_retriever.retrieve(query)
    #answer = [node.text for node in nodes]
    return nodes#answer

def graph_retriever(query, topk):
    
    retriever = graph_index.as_retriever(similarity_top_k=topk)
    nodes = retriever.retrieve(query)
    #answer = [node.text for node in nodes]

    return nodes #answer

def Hybrid_retriever(query, topk=25):
    retrievers = [graph_retriever, auto_retriever, get_bm25_retriever, window_retriever]# retriever1, auto_retriever, get_bm25_retriever, retriever2, auto_retriever
    
    nodes = []
    for retriever in retrievers:
        nodes.extend(retriever(query, topk))
    
    # combine the two lists of nodes
    all_nodes = []
    node_ids = set()
    for n in nodes:
        if n.node.node_id not in node_ids:
            all_nodes.append(n)
            node_ids.add(n.node.node_id)
    
    return all_nodes

## Reranker

In [150]:
import re
def get_rerank(answers, threshold=0.2):
    
    # 调用 get_similarity 函数计算相似度矩阵
    similarity_matrix = get_similarity(answers, answers)

    # 将每个句子的相似度之和保存到列表中，排除自相似（对角线）
    similarity_sums = np.sum(similarity_matrix, axis=1) - np.diag(similarity_matrix)
    #print(similarity_matrix)

     # 找到最大相似度之和
    max_similarity_sum = np.max(similarity_sums)

    # 计算每个句子与最大值的差值，并筛选出差值在 threshold 范围内的句子
    reranked_answer = [(answers[i], similarity_sums[i]) for i in range(len(answers))
                    if abs(similarity_sums[i] - max_similarity_sum) <= threshold]

    # 按 similarity_sums 从高到低排序，并提取句子
    result = [sentence for sentence, _ in sorted(reranked_answer, key=lambda x: x[1], reverse=True)]
    print(f"num of chunks: {len(result)}")
    return result

def get_rerank2(query, answers):
    
    # max_score = 0
    # final_answers = answers[0]
    # for answer in answers:
    #     score = get_scores(query, answer)
    #     if max_score > score:
    #         final_answers = answer
    #         max_score = score
    scores = get_scores(query, answers)
    idx = np.argmax(scores[0])
    print(scores[0][idx])
    final_answers = answers[idx]
            
    return [final_answers] 

def get_rerank3(query, answers):
    q = QueryBundle(query)
    nodes = answers
    # nodes = []
    # for i in answers:
    #     n = NodeWithScore(node=TextNode(text=i))
    #     nodes.append(n)
    nodes = [node for node in nodes if (not node.text.startswith('#')) and len(node.text)>40]
    reranked_nodes = reranker.postprocess_nodes(nodes, q)
    max_score = reranked_nodes[0].score
    # result = [reranked_nodes[0].text]
    
    result = []
    rerank_node=[]
    for node in reranked_nodes:
        text = node.text
        text = text.replace('. ','。').replace('! ','！').replace('? ','？').replace(', ','，').replace(' ','').strip()
        text = re.sub(r'(?<=[\u4e00-\u9fff])\n(?=[\u4e00-\u9fff])', '', text)
        if (node.score > max_score - 0.05 and text not in result):
            result.append(text)
            rerank_node.append(node)
            #print(node.score, text)
    
    return rerank_node[:1]

### Merge

In [151]:
def get_merge(text_list):
    """
    合并一个文本列表中的所有文本块，其中每个文本块的开头部分可能与前一个文本块的结尾部分重叠，
    或者每个文本块的结尾部分可能与下一个文本块的开头部分重叠。

    :param text_list: 文本列表
    :return: 合并后的文本
    """
    if not text_list:
        return ""

    # 初始化合并后的文本为第一个文本块
    merged_text = text_list[0]

    for i in range(1, len(text_list)):
        current_text = text_list[i]
        previous_text = merged_text
        
        # 检测 current_text 是否完全包含在 previous_text 中
        if current_text in previous_text:
            continue  # 跳过当前文本块，因为它已经被包含在合并后的文本中

        # 检测 previous_text 是否完全包含在 current_text 中
        if previous_text in current_text:
            merged_text = current_text  # 直接使用 current_text 替换合并后的文本
            continue

        # 检测 previous_text 结尾和 current_text 开头的重叠部分
        overlap_length_1 = 0
        for j in range(min(len(previous_text), len(current_text)), 0, -1):
            if previous_text[-j:] == current_text[:j]:
                overlap_length_1 = j
                break

        # 检测 current_text 结尾和 previous_text 开头的重叠部分
        overlap_length_2 = 0
        for j in range(min(len(previous_text), len(current_text)), 0, -1):
            if current_text[-j:] == previous_text[:j]:
                overlap_length_2 = j
                break

        # 选择重叠部分较长的情况进行合并
        if overlap_length_1 > overlap_length_2:
            # previous_text 结尾和 current_text 开头重叠
            merged_text += f"\n{current_text[overlap_length_1:]}"
        elif overlap_length_2 > overlap_length_1:
            # current_text 结尾和 previous_text 开头重叠
            merged_text = current_text + f"\n{merged_text[overlap_length_2:]}"
        else:
            # 没有找到重叠部分，或者重叠部分长度相等
            merged_text += f"\n{current_text}"

    return merged_text

### Pruning

In [167]:
def pruning(query, text):
    # 分割句子
    sentences = text.split('。')
    # 去除空字符串，如果有的话
    sentences = [s + '。' for s in sentences if s]  # 这里加上句号是为了保持输出格式的一致性
    
    subsequences = []
    
    # 生成所有连续子序列
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences) + 1):
            # 将连续的句子合并成一个字符串
            subsequence = ''.join(sentences[i:j]).strip()
            subsequences.append(subsequence)
            
    sentences2 = text.split('\n\n')
    # 去除空字符串，如果有的话
    sentences2 = [s + '\n\n' for s in sentences2 if s]  # 这里加上句号是为了保持输出格式的一致性
    
    # 生成所有连续子序列
    for i in range(len(sentences2)):
        for j in range(i + 1, len(sentences2) + 1):
            # 将连续的句子合并成一个字符串
            subsequence = ''.join(sentences2[i:j]).strip()
            subsequences.append(subsequence)
    
    #print(subsequences)
    q = QueryBundle(query)
    nodes = []
    for i in subsequences:
        n = NodeWithScore(node=TextNode(text=i))
        nodes.append(n)
    
    reranked_nodes = reranker.postprocess_nodes(nodes, q)
    
    # for n in reranked_nodes:
    #     print(n.text, n.score)
    return reranked_nodes[0].text, reranked_nodes[0].score

def pruning2(query, text):
    # 分割句子
    sentences = text.split('\n\n')
    # 去除空字符串，如果有的话
    sentences = [s + '\n\n' for s in sentences if s]  # 这里加上句号是为了保持输出格式的一致性
    
    subsequences = []
    # 生成所有连续子序列
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences) + 1):
            # 将连续的句子合并成一个字符串
            subsequence = ''.join(sentences[i:j]).strip()
            subsequences.append(subsequence)
            
    #print(subsequences)
    q = QueryBundle(query)
    nodes = []
    for i in subsequences:
        n = NodeWithScore(node=TextNode(text=i))
        nodes.append(n)
    
    reranked_nodes = reranker.postprocess_nodes(nodes, q)
    
    result = []
    seen_texts = []  # 创建一个集合来存储已见过的文本
    max_score = reranked_nodes[0].score
    final_text = reranked_nodes[0].text
    scores = []
    for n in reranked_nodes:
        if n.score > max_score - 0.05 and n.text not in result and (not n.text.startswith('#')) and len(n.text)>40:
            result.append(n.text)
            seen_texts.append(n)  # 将新添加的文本加入到集合中
            if len(n.text) < len(final_text):
                final_text = n.text
                
            #print(n.text, n.score)
    # for n in reranked_nodes:
    #     print(n.text, n.score)
    # result = result
    # text = '\n\n'.join(result)
    return final_text, reranked_nodes[0].score #reranked_nodes[0].text, reranked_nodes[0].score

def pruning3(query, text):
    
    sentences = text.split('。')
    # 去除空字符串，如果有的话
    sentences = [s + '。' for s in sentences if s]  # 这里加上句号是为了保持输出格式的一致性
    
    q = QueryBundle(query)
    nodes = []
    for i in sentences:
        n = NodeWithScore(node=TextNode(text=i))
        nodes.append(n)
    
    reranked_nodes = reranker.postprocess_nodes(nodes, q)
    
    a = []
    for node in reranked_nodes:
        if node.score > 0.99:
            
            a.append(node.text)
            
    if len(a)>1:
        return ''.join(a), reranked_nodes[0].score

    return pruning2(query, text)

### Processing data

In [168]:
import torch
torch.cuda.empty_cache()
import time
if __name__ == "__main__":
    question_df = pd.read_csv("B_question.csv")
    question_df = question_df
    sumbit_data = []
    for index, row in tqdm(question_df.iterrows()):
        ques_id = row['ques_id']
        query = row['question']
        start_time = time.time()
        candidate_answer = Hybrid_retriever(query)
        reranked_answer = get_rerank3(query, candidate_answer)
        reranked_text = [n.text for n in reranked_answer]
        merge_answer = get_merge(reranked_text)
        final_answer, score = pruning3(query, merge_answer)
        embedding = embed_model.get_text_embedding(final_answer)
        
        result = {
            'ques_id': ques_id,
            'question': query,
            'answer': final_answer,
            'embedding': embedding,
            'score': score
        }
        print(query)
        print(score)
        print(final_answer)
        print("========================")
        sumbit_data.append(result)
    sumbit_df = pd.DataFrame(sumbit_data)

0it [00:00, ?it/s]The tokenizer parameter is deprecated and will be removed in a future release. Use a stemmer from PyStemmer instead.
1it [00:03,  3.84s/it]

根据2023年年度报告摘要，中国联通在中央企业品牌建设能力对标中位于通信行业第几名？
0.9996056
2023年，中国联通坚持人民邮电为人民，聚焦高质量发展首要任务，不断深化业务融合、市场融通、能力融智，公司生产经营稳中有进，业务收入保持较快增长，用户规模创近年来新高，品牌影响力持续扩大，中国联通在中央企业品牌建设能力对标中，央企排名第四、通信行业位列第一。


The tokenizer parameter is deprecated and will be removed in a future release. Use a stemmer from PyStemmer instead.
2it [00:12,  6.38s/it]

帮我查一下，2023年联通5G中频基站的数量和新增量分别是多少？
0.9866736
坚持贯彻新发展理念，践行5G高质量发展。5G中频基站新增22万站，4G共享基站超过200万站。与友商在新疆启动全球首个5G异网漫游试商用，进一步提升边远地区网络资源使用效益；努力推动传输线路、管道光缆等基础设施的共建共享共维取得积极成效。以实际行动响应“双碳”政策，2023年节省运营开支约人民币390亿元，减少碳排放约1,150万吨，累计节省资本开支约人民币3,400亿元。


The tokenizer parameter is deprecated and will be removed in a future release. Use a stemmer from PyStemmer instead.
3it [00:18,  6.44s/it]

...

截至2024年7月19日，中国联通全网智算算力达到了多少？
0.9670405
中国联通网络部副总经理赵静宜指出，中国联通聚焦网络强国、数字中国主责，拓展联网通信、算网数智主业，联合产业链合作伙伴，加快算网基础设施布局。为进一步促进算网生态发展，中国联通重点构筑以下三方面能力。一是打造算力新供给，着力成为生态发展的“筑基者”。打造上海、呼和万卡智算集群，全网智算算力超15EFLOPS，发布AICC、AICP、星罗调度平台等五大智算产品，提供涵盖国家“东数西算”枢纽、31省重点城市、超600边缘节点的AIDC基座。


The tokenizer parameter is deprecated and will be removed in a future release. Use a stemmer from PyStemmer instead.
79it [07:20,  4.53s/it]

中国联通在智慧城市和数字乡村平台建设方面有哪些应用？
0.9955446
在智慧城市与数字乡村建设方面，联通云同样展现出了非凡的能力。服务超1400个智慧城市的经历，让联通云深刻理解了城市治理的复杂性和多样性。从智慧交通到智慧医疗，从智慧环保到智慧教育，联通云以其全面的解决方案和高效的实施能力，为城市管理者提供了强大的技术支持。同时，在数字乡村服务上，联通云也未曾缺席，覆盖超25万行政村的数字乡村服务，让偏远的农村地区也能享受到数字化带来的便利与福祉。这不仅缩小了城乡之间的数字鸿沟，更为乡村振兴战略的深入实施提供了有力支撑。


The tokenizer parameter is deprecated and will be removed in a future release. Use a stemmer from PyStemmer instead.
80it [07:24,  5.56s/it]

中国联通累计开通的4G和5G共享基站分别有多少站？
0.99922717
中国联通携手中国电信累计开通5G共享基站百万站, 开通4G共享基站155万站, 特别是快速新增了17万站900MHz低频打底网, 现已拥有117万个5G基站, 占到全球30%以上.


In [169]:
from datetime import datetime

# 获取当前时间并格式化
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

# 构建新的文件名
filename = f"./submit/submit_{current_time}.csv"
filename_s = f"./submit/submit_{current_time}_score.csv"

# 将DataFrame保存为CSV文件
sumbit_df.to_csv(filename_s, index=False, encoding='utf-8')

sumbit_df = sumbit_df.drop(columns=['score'])
sumbit_df.to_csv(filename, index=False, encoding='utf-8')

In [165]:
query = "有效提升通信网络覆盖深度广度的措施有哪些？"
candidate_answer = Hybrid_retriever(query)
reranked_answer = get_rerank3(query, candidate_answer)
reranked_text = [n.text for n in reranked_answer]
merge_answer = get_merge(reranked_text)
final_answer = pruning3(query, merge_answer)
final_answer

The tokenizer parameter is deprecated and will be removed in a future release. Use a stemmer from PyStemmer instead.


1


('#（一）有效提升通信网络覆盖深度广度\n\n加快推进“双千兆”网络建设。', 0.9829549)

In [143]:
len("#（一）有效提升通信网络覆盖深度广度\n\n加快推进“双千兆”网络建设。")

34

In [163]:
merge_answer

'迫切需要把握宽带体系化演进趋势，立足长远做好谋篇布局，发挥政府和市场、中央和地方、国有资本和社会资本的多轮驱动作用，适度超前推动宽带设施演进升级，建用互促深化融合创新，供需互动提高基础设施全生命周期综合效益，不断巩固提升既有优势，加快打通经济社会发展信息“大动脉”。\n\n#（一）有效提升通信网络覆盖深度广度\n\n加快推进“双千兆”网络建设。'

In [35]:
final_answer

'坚持贯彻新发展理念，践行5G高质量发展。5G中频基站新增22万站，4G共享基站超过200万站。与友商在新疆启动全球首个5G异网漫游试商用，进一步提升边远地区网络资源使用效益；努力推动传输线路、管道光缆等基础设施的共建共享共维取得积极成效。以实际行动响应“双碳”政策，2023年节省运营开支约人民币390亿元，减少碳排放约1,150万吨，累计节省资本开支约人民币3,400亿元。'

In [161]:
from llama_index.core.response.notebook_utils import display_source_node

for retrieved_node in reranked_answer:
    print(retrieved_node.get_content(metadata_mode="all"))
    display_source_node(retrieved_node, source_length=1000)

file_path: /mnt/workspace/基于运营商文本数据的知识库检索/B_document/BZ02.pdf
file_name: BZ02.pdf
file_type: application/pdf
file_size: 4589584
creation_date: 2024-11-14
last_modified_date: 2024-08-19
发布时间: 
发布人:

迫切需要把握宽带体系化演进趋势，立足长远做好谋篇布局，发挥政府和市场、中央和地方、国有资本和社会资本的多轮驱动作用，适度超前推动宽带设施演进升级，建用互促深化融合创新，供需互动提高基础设施全生命周期综合效益，不断巩固提升既有优势，加快打通经济社会发展信息“大动脉”。

#（一）有效提升通信网络覆盖深度广度

加快推进“双千兆”网络建设。


**Node ID:** 7f38f704-ee6a-4b8f-a4b8-a0b47e53a22d<br>**Similarity:** 0.9719997644424438<br>**Text:** 迫切需要把握宽带体系化演进趋势，立足长远做好谋篇布局，发挥政府和市场、中央和地方、国有资本和社会资本的多轮驱动作用，适度超前推动宽带设施演进升级，建用互促深化融合创新，供需互动提高基础设施全生命周期综合效益，不断巩固提升既有优势，加快打通经济社会发展信息“大动脉”。

#（一）有效提升通信网络覆盖深度广度

加快推进“双千兆”网络建设。<br>

In [77]:
reranked_answer[0].metadata

{'file_path': '/mnt/workspace/基于运营商文本数据的知识库检索/B_document/BY01.pdf',
 'file_name': 'BY01.pdf',
 'file_type': 'application/pdf',
 'file_size': 821719,
 'creation_date': '2024-11-14',
 'last_modified_date': '2024-08-12',
 '发布时间': '',
 '发布人': '',
 'document_title': '中国联通2023年：业绩稳健攀升，业务全面升级报告',
 'questions_this_excerpt_can_answer': '1. 中国联通在2023年采取了哪些措施以响应“双碳”政策，并在节能减排方面取得了哪些成效？\n2. 中国联通在2023年如何通过科技创新推动产业创新，并具体提升了哪些方面的能力？\n3. 中国联通在提升社会责任和公司治理方面采取了哪些具体措施，并在哪些方面获得了行业认可？',
 'excerpt_keywords': '5G发展, 节能减排, 科技创新, 产业升级, 资本运营, 人工智能, 大模型, 数据中心, 社会责任, 公司治理'}

In [69]:
print(reranked_text[0])

坚持贯彻新发展理念，践行5G高质量发展。5G中频基站新增22万站，4G共享基站超过200万站。与友商在新疆启动全球首个5G异网漫游试商用，进一步提升边远地区网络资源使用效益；努力推动传输线路、管道光缆等基础设施的共建共享共维取得积极成效。以实际行动响应“双碳”政策，2023年节省运营开支约人民币390亿元，减少碳排放约1,150万吨，累计节省资本开支约人民币3,400亿元。

#创新驱动，深化转型蹄疾步稳

公司积极锻造核心能力，持续强劲科技创新引擎，着力以科技创新推动产业创新，加快发展新质生产力。加大人才培育力度，科技创新人员占比达到40%；年内授权专利达到2,287件，同比增长37%。

#中国联合网络通信股份有限公司

#2023年年度报告

提升资本运营效能，加大资本合作力度，设立云宙时代、联通数智医疗科技等合资公司，专精特新数字化服务能力进一步提升。

公司将人工智能作为重要战略方向，加快布局人工智能新兴产业，结合中国联通在算网基础设施、丰富业务场景、广泛客户触点的优势，着力打造联通特色的大模型开发应用范式。在刚刚过去的世界移动通信大会上，公司推出“元景”大模型体系，根据不同功能需求布局多参数版本，推动大模型更好赋能实体经济应用场景。以AI+大数据全面提升客户服务水平，相关成果入选行业权威奖项“吴文俊人工智能科学技术奖”。

进一步加快卫星通信统筹布局，积极推动天地一体通信产业化、体系化发展。

#（五）社会责任和公司治理

公司胸怀国之大者，主动服务国家战略，把经济责任与社会责任有机结合。过去一年，公司在环境、社会、公司治理等方面彰显担当，为满足人民群众美好信息生活需要和千行百业数字化网络化智能化需求贡献联通力量。公司持续锻造坚实可靠的安全网络，提升数据与信息安全能力，健全协同保障体系，筑牢国家网络安全屏障。践行“双碳”战略，积极打造技术先进、绿色低碳、安全可靠的新型数据中心。构建普惠便捷的数字社会，全面助力乡村振兴，数字乡村平台覆盖25.5万个行政村，服务逾2,600万村民。充分展现责任担当，在通信重保、抢险救灾、改革攻坚等大事要事中践行初心使命，顺利完成抢险救灾通信保障任务。持续提升上市公司质量，通过反向路演等活动加强与资本市场沟通交流，积极回应股东关切，推动内在价值与市场价值共同提升。

公司持续完善治理机制，为持续健康发展提供坚实保障，连续获中国